# 6 - torchtext的使用

虽然在输入模型前，数据处理的基本思路都是分词、清洗、数学表示，但是神经网络对于数据结构的要求和传统机器学习不太一样，主要表现为神经网络在采取`批量梯度下降法`(BGD)时，需要小批量地输入数据，同时其中对`训练/验证/测试集`概念使用的也更加频繁。对此，pytorch中的`torchtext`库提供了一套**完整**的文本数据集处理机制，使用起来非常方便，而且也可以应用于除pytorch外的框架中（如tensorflow/keras）。

几个核心概念：
+ **Field**：`域`，一步解决`分词、清洗、表示`的问题
+ **Dataset**：`数据集`，读取数据文件，可进行split操作
+ **Iterator**：`迭代器`，由dataset得到，为最终输入，同样可以进行split操作

数据流：

    data -> Dataset -> Dataloader/Iterator

基本过程：
1. 定义Field实例对象
2. 利用数据生成examples得到Dataset，该过程调用Field.preprocess，包括：tokenize -> pipeline -> preprocessing
3. 基于Dataset构建vocab词典（build_vocab），可以传入词的向量表示（vectors）
4. 基于Dataset得到Iterator，在构建batch的过程中调用Field.process，包括：pad -> numericalize -> tensor（最终产物）

In [18]:
import os
import re
import traceback
import pickle

import torch
from torchtext import data, vocab
from torchtext.data import Dataset, TabularDataset
from torchtext.data import Iterator, BucketIterator

### Field
以下是构建`Field`实例对象时可传入的所有参数：

   **Field**(**sequential**=True, **use_vocab**=True, init_token=None, eos_token=None, **fix_length**=None, **dtype**=torch.int64, **preprocessing**=None, postprocessing=None, **lower**=False, **tokenize**=None, tokenizer_language='en', include_lengths=False, **batch_first**=False, pad_token='<pad>', unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False)

其中：
+ **sequential**：指单条数据是否是序列的（如一句话可看作一个序列），为`label`构建Field时将其指定为**False**
+ **use_vocab**：如果label数据已经是整形，则设置use_vocab=False
+ **fix_length**：是否使用padding/cut指定序列长度
+ **preprocessing**：`"The Pipeline that will be applied to examples using this field after tokenizing but before numericalizing"`. -> 可传入一个preprocessor，介于分词与表示之间，用于`清洗等`其他操作
+ **lower**：`str.lower()`
+ **tokenize**：传入分词器
+ **batch_first**：是否将batch作为数据的第一个维度
+ **init_token/eos_token/pad_token/unk_token**：特殊符号的表示
    
Field定义了如何处理数据，其中两个核心方法为`preprocess`和`process`。注意Field只是**`规则`**的指定，只有当数据传入后才能构建实体词典。
    
如果要使用现成的模型，则需要沿用其Field的构建，具体方法参见*。

In [19]:
def tokenizer(text):
    text = text.lower()
    text = re.sub(r',|\.|\?|!','',text)
    tokens = text.split()
    tokens = list(filter(lambda x:len(x)>0,tokens))
    if len(tokens)<5:
        tokens.extend([' ']*(5-len(tokens)))
    return tokens

# 设定text和label的Field
text_field = data.Field(lower=True,tokenize=tokenizer,batch_first=True)
label_field = data.Field(sequential=False,batch_first=True)

### Dataset
+ Dataset由examples组成【examples必须是data.Example类型】
+ 一个example由`text和label`组成，分别对应data中的X和y。
+ X->text, 以及y->label要经过`Field`的处理（映射）
+ data.Dataset是终极父类，如果要`自定义dataset`从它`继承`
+ data.`TabularDataset`(继承类)可以用来读取存储在"CSV"，"TSV"或"JSON"中的数据
+ dataset提供`splits类方法`用于同时生成train/validation/test数据的dataset，但是需要指定各自的文件路径

In [20]:
train_dataset, test_dataset = TabularDataset.splits(
        path='processed_data', format='csv', skip_header=True,
        train='best3_train.csv', test='best3_test.csv',
        fields=[('text',text_field),('label',label_field)])
# 注意！
# fields=[f1,f2,...]严格按照csv中的列顺序，'text'和'label'是对每列数据重新赋予的名字
# 在传入前应该先将index列除去

json或许是比csv/tcs更好的选择，因为：1.可以存储列表 2.无需担心tab等符号使得列读取混乱

### build_vocab
根据具体的数据集构建Vocab对象 -> `Field.build_vocab` -> `Field.vocab`

Field.vocab对象中，如果不指定vectors，则依据词频构建词典（得到vocab.itos\/stoi）。否则使用指定的vectors。

vocab.vectors的构造有两种方式：
1. 利用库中自带的向量模型

  可选的预训练模型有`"charngram.100d", "fasttext.en.300d", "fasttext.simple.300d", "glove.42B.300d", "glove.840B.300d", "glove.twitter.27B.25d", "glove.twitter.27B.50d", "glove.twitter.27B.100d", "glove.twitter.27B.200d", "glove.6B.50d", "glove.6B.100d", "glove.6B.200d", "glove.6B.300d"`。其中d表示向量长度。

2. 传入外部训练的向量模型

In [21]:
# 1.利用自带的pretrain模型，注意下载可能需要比较久的时间
# text_field.build_vocab(train_dataset,test_dataset,vectors='glove.6B.200d')

# 2.外部传入向量模型
# vectors = vocab.Vectors(name=embedding_model_path,cache=cache_path)
# name为传入模型的路径，cache为向量数据缓存的路径，默认在.vector_cache文件夹下

# 此处直接使用索引而非向量
text_field.build_vocab(train_dataset,test_dataset)
label_field.build_vocab(train_dataset,test_dataset)

实不相瞒，我从来没有下成功过。

<img src='images/memes/RuntimeError.jpg'>

### Iterator
迭代器，用于生成batch。同样使用spilts方法来定义训练/验证/测试集。

下面以BucketIterator为例，其与普通迭代器的区别在于，它会将长度相似的数据放在一个batch中。

In [22]:
train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset,test_dataset),
    sort_key=lambda x:len(x.text),shuffle=True,
    batch_sizes=(64,int(len(test_dataset)/16)))

# 至此，观察数据的内部排列方式
for batch in train_iter:
    feature, target = batch.text, batch.label
    print(feature.shape)
    print(target.shape)
    print(feature[0])
    break

torch.Size([64, 62])
torch.Size([64])
tensor([    6, 10018,   393,     2,     2,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])


流程回顾：

1. 定义Field实例对象
2. 利用Dataset生成examples，该过程调用Field.preprocess，包括：tokenize -> pipeline -> preprocessing
3. 基于Dataset构建vocab词典（build_vocab），可以传入词的向量表示（vectors）
4. 基于Dataset得到Iterator，在构建batch的过程中调用Field.process，包括：pad -> numericalize -> tensor（最终产物）